# Cloning and Building Darknet


In [ ]:
%cd /content/
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
# make darknet (build)
!make

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

# Uploading Local or Google Drive Files to Use

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

# Download pre-trained weights for the convolutional layers.
This step downloads the weights for the convolutional layers of the Tiny-YOLOv4 network. By using these weights it helps the object detector to be more accurate and not have to train as long.

In [ ]:
# upload pretrained convolutional layer weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

# Copy contents from Gdrive

In [ ]:
!cp /content/gdrive/MyDrive/thermal-v20/train.zip /content/darknet/data/

In [ ]:
!cp /content/gdrive/MyDrive/optical-v7-hybrid/test.zip /content/darknet/data/

In [ ]:
%cd /content/darknet/data/
!unzip train.zip

In [ ]:
!unzip test.zip

In [ ]:
%cd /content/darknet/
%pwd

# Train the Custom Model

In [ ]:
# train your custom detector
!./darknet detector train /content/darknet/data/obj.data /content/darknet/cfg/tiny-yolov4-v19-final.cfg /content/gdrive/MyDrive/thermal-v20/tiny-yolov4-v19-final_6000.weights -dont_show

Chart of how the model did throughout the training process It shows a chart of the average loss vs. iterations.

In [ ]:
imShow('chart.png')

In [ ]:
!./darknet detector map /content/darknet/data/obj.data /content/darknet/cfg/tiny-yolov4-v19-final.cfg /content/gdrive/MyDrive/thermal-v20/tiny-yolov4-v19-final_last.weights -ext_output < data/test.txt > map.txt

In [ ]:
!./darknet detector map /content/darknet/data/obj.data /content/darknet/cfg/tiny-yolov4-optical-v7-hybrid.cfg /content/gdrive/MyDrive/optical-v7-hybrid/tiny-yolov4-optical-v7-hybrid_last.weights -ext_output < data/test.txt > optical-v7-result-test.txt

In [ ]:
!cp /content/darknet/map.txt /content/gdrive/MyDrive/thermal-v20/

# Run Custom Object Detector!!!


## Run for Images

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test /content/darknet/data/obj.data /content/darknet/cfg/yolo-obj.cfg /content/darknet/backup/yolo-obj_last.weights /content/darknet/data/l1.png -thresh 0.35
imShow('predictions.jpg')

## Run for Videos

In [ ]:
!./darknet detector demo data/obj.data cfg/yolo-obj-V5.cfg /content/gdrive/MyDrive/thermal-pinhole-leak-det/yolo-obj-V5_1000.weights /content/gdrive/MyDrive/testing-videos/set1-east-west.mp4 -dont_show set1-east-west.mp4 -i 0 -out_filename /content/gdrive/MyDrive/thermal-pinhole-leak-det/set1-east-west-res.avi -thresh 0.3